In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (csv location)
emission_data_csv = "Resources/historical_emissions_sectors.csv"

# Read in csv file from Resources Folder
emission_data = pd.read_csv(emission_data_csv)
emission_data.head()

,Country,Data source,Sector,Gas,Unit,2019,2018,2017,2016,2015,...,1969,1968,1967,1966,1965,1964,1963,1962,1961,1960
0,World,GCP,Total fossil fuels and cement,CO2,MtCO₂e,36440.76,36420.16,35694.69,35222.03,35211.04,...,13692.37,12834.99,12171.81,11794.42,11270.46,10768.50,10237.22,9687.62,9354.19,9335.87
1,World,GCP,Coal,CO2,MtCO₂e,14362.56,14619.36,14414.18,14362.88,14623.02,...,5503.33,5301.81,5243.18,5327.46,5279.82,5195.55,5100.29,4902.43,4917.09,5129.60
2,World,GCP,Oil,CO2,MtCO₂e,12354.11,12252.42,12175.47,11970.29,11915.33,...,5964.99,5514.32,5063.65,4748.54,4411.46,4100.02,3806.90,3554.08,3304.93,3121.73
3,China,GCP,Total fossil fuels and cement,CO2,MtCO₂e,10174.68,9956.57,9750.73,9552.52,9683.20,...,575.94,467.81,432.22,521.46,474.68,435.70,435.52,439.34,550.96,778.98
4,World,GCP,Gas,CO2,MtCO₂e,7615.71,7489.22,7115.49,6939.62,6763.74,...,1711.09,1564.53,1436.29,1341.02,1234.77,1154.16,1047.90,967.30,883.02,835.39


In [2]:
#delete the extra columns [Data Source, Gas, Unit]
emission_data = emission_data.drop(columns = ['Data source','Gas','Unit'])
emission_data.head()

,Country,Sector,2019,2018,2017,2016,2015,2014,2013,2012,...,1969,1968,1967,1966,1965,1964,1963,1962,1961,1960
0,World,Total fossil fuels and cement,36440.76,36420.16,35694.69,35222.03,35211.04,35244.02,34987.54,34760.37,...,13692.37,12834.99,12171.81,11794.42,11270.46,10768.50,10237.22,9687.62,9354.19,9335.87
1,World,Coal,14362.56,14619.36,14414.18,14362.88,14623.02,14945.46,14919.81,14901.49,...,5503.33,5301.81,5243.18,5327.46,5279.82,5195.55,5100.29,4902.43,4917.09,5129.60
2,World,Oil,12354.11,12252.42,12175.47,11970.29,11915.33,11743.12,11629.54,11523.28,...,5964.99,5514.32,5063.65,4748.54,4411.46,4100.02,3806.90,3554.08,3304.93,3121.73
3,China,Total fossil fuels and cement,10174.68,9956.57,9750.73,9552.52,9683.20,9820.36,9796.53,9633.90,...,575.94,467.81,432.22,521.46,474.68,435.70,435.52,439.34,550.96,778.98
4,World,Gas,7615.71,7489.22,7115.49,6939.62,6763.74,6591.54,6540.24,6496.27,...,1711.09,1564.53,1436.29,1341.02,1234.77,1154.16,1047.90,967.30,883.02,835.39


In [3]:
#Locate only the total emission
filtered_emission_total = emission_data.loc[emission_data["Sector"] == 'Total fossil fuels and cement']
filtered_emission_total = filtered_emission_total.drop(columns = ['Sector'])
filtered_emission_total

,Country,2019,2018,2017,2016,2015,2014,2013,2012,2011,...,1969,1968,1967,1966,1965,1964,1963,1962,1961,1960
0,World,36440.76,36420.16,35694.69,35222.03,35211.04,35244.02,34987.54,34760.37,34210.77,...,13692.37,12834.99,12171.81,11794.42,11270.46,10768.50,10237.22,9687.62,9354.19,9335.87
3,China,10174.68,9956.57,9750.73,9552.52,9683.20,9820.36,9796.53,9633.90,9388.20,...,575.94,467.81,432.22,521.46,474.68,435.70,435.52,439.34,550.96,778.98
6,United States,5284.70,5424.88,5253.61,5292.27,5412.43,5561.72,5514.03,5367.57,5565.29,...,4021.50,3828.31,3693.30,3559.18,3388.19,3252.76,3116.02,2984.00,2877.37,2887.82
7,European Union (27),2916.91,3054.49,3127.50,3103.52,3098.97,3044.69,3179.99,3260.93,3336.52,...,3150.78,2949.08,2777.13,2715.40,2655.79,2611.54,2500.41,2312.45,2157.63,2067.81
8,India,2616.45,2591.32,2456.85,2392.36,2253.43,2184.44,2033.40,1941.31,1766.07,...,177.64,174.30,159.78,159.59,153.92,139.69,142.65,132.78,120.58,111.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,Cook Islands,0.08,0.07,0.07,0.06,0.06,0.07,0.06,0.06,0.06,...,0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
808,Kiribati,0.07,0.07,0.07,0.07,0.06,0.06,0.05,0.05,0.05,...,0.01,0.03,0.02,0.01,0.01,0.02,0.02,0.01,0.01,NaN
814,Nauru,0.05,0.05,0.05,0.04,0.05,0.05,0.05,0.04,0.04,...,0.07,0.05,0.05,0.03,0.03,0.03,NaN,NaN,NaN,NaN
832,Niue,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
filtered_emission_total = filtered_emission_total.set_index('Country')
filtered_emission_total.head()

,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,...,1969,1968,1967,1966,1965,1964,1963,1962,1961,1960
Country,,,,,,,,,,,,,,,,,,,,,
World,36440.76,36420.16,35694.69,35222.03,35211.04,35244.02,34987.54,34760.37,34210.77,33133.55,...,13692.37,12834.99,12171.81,11794.42,11270.46,10768.50,10237.22,9687.62,9354.19,9335.87
China,10174.68,9956.57,9750.73,9552.52,9683.20,9820.36,9796.53,9633.90,9388.20,8500.54,...,575.94,467.81,432.22,521.46,474.68,435.70,435.52,439.34,550.96,778.98
United States,5284.70,5424.88,5253.61,5292.27,5412.43,5561.72,5514.03,5367.57,5565.29,5698.06,...,4021.50,3828.31,3693.30,3559.18,3388.19,3252.76,3116.02,2984.00,2877.37,2887.82
European Union (27),2916.91,3054.49,3127.50,3103.52,3098.97,3044.69,3179.99,3260.93,3336.52,3442.04,...,3150.78,2949.08,2777.13,2715.40,2655.79,2611.54,2500.41,2312.45,2157.63,2067.81
India,2616.45,2591.32,2456.85,2392.36,2253.43,2184.44,2033.40,1941.31,1766.07,1678.48,...,177.64,174.30,159.78,159.59,153.92,139.69,142.65,132.78,120.58,111.49


In [5]:
filtered_emission_total = filtered_emission_total.drop(['World','European Union (27)'])
filtered_emission_total

,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,...,1969,1968,1967,1966,1965,1964,1963,1962,1961,1960
Country,,,,,,,,,,,,,,,,,,,,,
China,10174.68,9956.57,9750.73,9552.52,9683.20,9820.36,9796.53,9633.90,9388.20,8500.54,...,575.94,467.81,432.22,521.46,474.68,435.70,435.52,439.34,550.96,778.98
United States,5284.70,5424.88,5253.61,5292.27,5412.43,5561.72,5514.03,5367.57,5565.29,5698.06,...,4021.50,3828.31,3693.30,3559.18,3388.19,3252.76,3116.02,2984.00,2877.37,2887.82
India,2616.45,2591.32,2456.85,2392.36,2253.43,2184.44,2033.40,1941.31,1766.07,1678.48,...,177.64,174.30,159.78,159.59,153.92,139.69,142.65,132.78,120.58,111.49
Russia,1678.37,1691.36,1646.18,1618.30,1622.86,1622.35,1619.17,1679.87,1664.95,1612.88,...,1371.48,1316.89,1280.90,1224.98,1159.99,1093.52,1028.22,958.76,913.53,885.86
Japan,1106.66,1135.69,1187.66,1203.17,1222.78,1263.05,1314.70,1305.43,1264.16,1214.07,...,653.22,561.91,489.31,419.25,386.47,358.89,324.84,292.86,282.79,232.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cook Islands,0.08,0.07,0.07,0.06,0.06,0.07,0.06,0.06,0.06,0.05,...,0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Kiribati,0.07,0.07,0.07,0.07,0.06,0.06,0.05,0.05,0.05,0.06,...,0.01,0.03,0.02,0.01,0.01,0.02,0.02,0.01,0.01,NaN
Nauru,0.05,0.05,0.05,0.04,0.05,0.05,0.05,0.04,0.04,0.04,...,0.07,0.05,0.05,0.03,0.03,0.03,NaN,NaN,NaN,NaN


In [6]:
filtered_emission_total = filtered_emission_total.dropna()
filtered_emission_total

,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,...,1969,1968,1967,1966,1965,1964,1963,1962,1961,1960
Country,,,,,,,,,,,,,,,,,,,,,
China,10174.68,9956.57,9750.73,9552.52,9683.20,9820.36,9796.53,9633.90,9388.20,8500.54,...,575.94,467.81,432.22,521.46,474.68,435.70,435.52,439.34,550.96,778.98
United States,5284.70,5424.88,5253.61,5292.27,5412.43,5561.72,5514.03,5367.57,5565.29,5698.06,...,4021.50,3828.31,3693.30,3559.18,3388.19,3252.76,3116.02,2984.00,2877.37,2887.82
India,2616.45,2591.32,2456.85,2392.36,2253.43,2184.44,2033.40,1941.31,1766.07,1678.48,...,177.64,174.30,159.78,159.59,153.92,139.69,142.65,132.78,120.58,111.49
Russia,1678.37,1691.36,1646.18,1618.30,1622.86,1622.35,1619.17,1679.87,1664.95,1612.88,...,1371.48,1316.89,1280.90,1224.98,1159.99,1093.52,1028.22,958.76,913.53,885.86
Japan,1106.66,1135.69,1187.66,1203.17,1222.78,1263.05,1314.70,1305.43,1264.16,1214.07,...,653.22,561.91,489.31,419.25,386.47,358.89,324.84,292.86,282.79,232.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Saint Vincent and the Grenadines,0.26,0.26,0.26,0.25,0.21,0.27,0.21,0.25,0.20,0.22,...,0.03,0.02,0.02,0.02,0.01,0.02,0.01,0.01,0.01,0.01
Comoros,0.25,0.25,0.24,0.20,0.17,0.15,0.18,0.14,0.14,0.16,...,0.02,0.02,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01
Tonga,0.18,0.16,0.16,0.13,0.12,0.11,0.11,0.11,0.10,0.12,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01


In [59]:
emission_transposed = filtered_emission_total.T
emission_transposed.head()

Country,China,United States,India,Russia,Japan,Iran,Germany,Indonesia,South Korea,Saudi Arabia,...,Guinea-Bissau,Solomon Islands,Central African Republic,Grenada,Samoa,Saint Vincent and the Grenadines,Comoros,Tonga,Dominica,Sao Tome and Principe
2019,10174.68,5284.70,2616.45,1678.37,1106.66,779.53,701.96,617.51,611.26,582.15,...,0.32,0.32,0.31,0.29,0.29,0.26,0.25,0.18,0.16,0.13
2018,9956.57,5424.88,2591.32,1691.36,1135.69,755.40,755.36,576.58,634.93,576.76,...,0.31,0.30,0.30,0.28,0.27,0.26,0.25,0.16,0.17,0.13
2017,9750.73,5253.61,2456.85,1646.18,1187.66,724.58,786.65,531.01,620.61,579.42,...,0.29,0.29,0.30,0.28,0.26,0.26,0.24,0.16,0.16,0.12
2016,9552.52,5292.27,2392.36,1618.30,1203.17,648.06,800.51,568.19,617.96,565.75,...,0.29,0.29,0.30,0.27,0.25,0.25,0.20,0.13,0.18,0.12
2015,9683.20,5412.43,2253.43,1622.86,1222.78,640.81,795.82,507.01,595.39,645.41,...,0.28,0.29,0.29,0.26,0.23,0.21,0.17,0.12,0.18,0.11


In [66]:
bins = [1960,1969,1979,1989,1999,2009,2019]
decade_name = ['1960s','1970s','1980s','1990s','2000s','2010s']

emission_transposed = emission_transposed.reset_index()
print(emission_transposed)

# emission_transposed = emission_transposed.astype({'index': 'float64'}).dtypes

china = pd.cut(emission_transposed["index"],bins, decade_name)
china

#bins = [1960,1969,1970,1979,1980,1989,1990,1999,2000,2009,2010,2019]

KeyError: "None of [Index(['China'], dtype='object')] are in the [columns]"

In [ ]:
# sum_emission_dummy = []
# sum_emission = []  #{"China":[sum decade1,sum decade2]}
# year_index = 0

In [ ]:
# #outer loop to iterate the years (incremented by 10)
# for year in range(2019,1960,-10):
#     #inner loop to iterate each row (country)
#     for row in filtered_emission_total.iterrows():
#         #third loop to iterate through the decade
#         for decade in range(year_index,year_index+10):     
#                                         #a dummy df with 10 numbers for each row (country)
#                                         #locate where column name >= year AND column name < year+10
#                                             #(row[f'{decade}'] >= year) & (row[f'{decade}'] < (year+10))
#             sum_emission_dummy.append(row[1][decade])
            
#         #sum the dummy df to get a decade sum for each row (country)    
#         sum_dummy_df = pd.DataFrame(sum_emission_dummy)
#         sum_emission_dummy = []

#         row_group_sum = sum_dummy_df.sum()
        
#         #input the sum for the row (country) per decade
#         sum_emission.append(row_group_sum)

#     #increment index by 10 to get columns for next decade
#     year_index += 10
    
# sum_emission

In [ ]:
print('hello')